##ThinkDSP

This notebook contains code examples from Chapter 7: Discrete Fourier Transform

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
from __future__ import print_function, division

import thinkdsp
import thinkplot
import thinkstats2

import numpy as np

import warnings
warnings.filterwarnings('ignore')

PI2 = 2 * np.pi

np.set_printoptions(precision=3, suppress=True)
%matplotlib inline

Here's the definition of ComplexSinusoid, with print statements to display intermediate results.

In [ ]:
class ComplexSinusoid(thinkdsp.Sinusoid):
    """Represents a complex exponential signal."""
    
    def evaluate(self, ts):
        """Evaluates the signal at the given times.

        ts: float array of times
        
        returns: float wave array
        """
        print(ts)
        phases = PI2 * self.freq * ts + self.offset
        print(phases)
        ys = self.amp * np.exp(1j * phases)
        return ys

Here's an example:

In [ ]:
signal = ComplexSinusoid(freq=1, amp=0.6, offset=1)
wave = signal.make_wave(duration=1, framerate=4)
print(wave.ys)

The simplest way to synthesize a mixture of signals is to evaluate the signals and add them up.

In [ ]:
def synthesize1(amps, freqs, ts):
    components = [thinkdsp.ComplexSinusoid(freq, amp)
                  for amp, freq in zip(amps, freqs)]
    signal = thinkdsp.SumSignal(*components)
    ys = signal.evaluate(ts)
    return ys

Here's an example that's a mixture of 4 components.

In [ ]:
amps = np.array([0.6, 0.25, 0.1, 0.05])
freqs = [100, 200, 300, 400]
framerate = 11025

ts = np.linspace(0, 1, framerate)
ys = synthesize1(amps, freqs, ts)
print(ys)

Now we can plot the real and imaginary parts:

In [ ]:
print(np.fft.fft(ys))

The inverse DFT is almost the same, except we don't have to transpose $M$ and we have to divide through by $N$.

In [ ]:
def idft(ys):
    N = len(ys)
    M = synthesis_matrix(N)
    amps = M.dot(ys) / N
    return amps

We can confirm that `dft(idft(amps))` yields `amps`:

In [ ]:
ys = idft(amps)
print(dft(ys))

###Real signals

Let's see what happens when we apply DFT to a real-valued signal.

In [ ]:
framerate = 10000
signal = thinkdsp.SawtoothSignal(freq=500)
wave = signal.make_wave(duration=0.1, framerate=framerate)
wave.make_audio()

`wave` is a 500 Hz sawtooth signal sampled at 10 kHz.

In [ ]:
hs = dft(wave.ys)
len(wave.ys), len(hs)

`hs` is the DFT of this wave, and `amps` contains the amplitudes.

In [ ]:
amps = np.absolute(hs)
thinkplot.plot(amps)
thinkplot.config(xlabel='Frequency (unspecified units)', ylabel='DFT')

The DFT assumes that the sampling rate is N per time unit, for an arbitrary time unit.  We have to convert to actual units -- seconds -- like this:

In [ ]:
N = len(hs)
fs = np.arange(N) * framerate / N

Also, the DFT of a real signal is symmetric, so the right side is redundant.  Normally, we only compute and plot the first half:

In [ ]:
thinkplot.plot(fs[:N//2+1], amps[:N//2+1])
thinkplot.config(xlabel='Frequency (Hz)', ylabel='DFT')

Let's get a better sense for why the DFT of a real signal is symmetric.  I'll start by making the inverse DFT matrix for $N=8$.

In [ ]:
wave = thinkdsp.TriangleSignal(freq=1).make_wave(duration=1, framerate=8)
wave.ys

Here's what the wave looks like.

In [ ]:
wave.plot()

Now let's look at rows 3 and 5 of the DFT matrix:

In [ ]:
row3 = Mstar[3, :]
print(row3)

In [ ]:
row5 = Mstar[5, :]
row5

They are almost the same, but row5 is the complex conjugate of row3.

In [ ]:
def approx_equal(a, b, tol=1e-10):
    return sum(abs(a-b)) < tol

In [ ]:
approx_equal(row3, row5.conj())

When we multiply the DFT matrix and the wave array, the element with index 3 is:

In [ ]:
X3 = row3.dot(wave.ys)
X3

And the element with index 5 is:

In [ ]:
n = 500
thinkplot.plot(ts[:n], ys[:n].real)
thinkplot.plot(ts[:n], ys[:n].imag)

The real part is a mixture of cosines; the imaginary part is a mixture of sines.  They contain the same frequency components with the same amplitudes, so they sound the same to us:

In [ ]:
wave = thinkdsp.Wave(ys.real, framerate)
wave.apodize()
wave.make_audio()

In [ ]:
wave = thinkdsp.Wave(ys.imag, framerate)
wave.apodize()
wave.make_audio()

We can express the same process using matrix multiplication.

In [ ]:
def synthesize2(amps, freqs, ts):
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    ys = np.dot(M, amps)
    return ys

And it should sound the same.

In [ ]:
amps = np.array([0.6, 0.25, 0.1, 0.05])
ys = synthesize2(amps, freqs, ts)
print(ys)

To see the effect of a complex amplitude, we can rotate the amplitudes by 1.5 radian:

In [ ]:
phi = 1.5
amps2 = amps * np.exp(1j * phi)
ys2 = synthesize2(amps2, freqs, ts)

n = 500
thinkplot.plot(ts[:n], ys.real[:n], label=r'$\phi_0 = 0$')
thinkplot.plot(ts[:n], ys2.real[:n], label=r'$\phi_0 = 1.5$')
thinkplot.config(ylim=[-1.15, 1.05], loc='lower right')

Rotating all components by the same phase offset changes the shape of the waveform because the components have different periods; a constant offset has a different effect on each component.

###Analysis

The simplest way to analyze a signal---that is, find the amplitude for each component---is to create the same matrix we used for synthesis and then solve the system of linear equations.

In [ ]:
def analyze1(ys, freqs, ts):
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = np.linalg.solve(M, ys)
    return amps

Using the first 4 values from the wave array, we can recover the amplitudes.

In [ ]:
n = len(freqs)
amps2 = analyze1(ys[:n], freqs, ts[:n])
print(amps2)

If we define the `freqs` from 0 to N-1 and `ts` from 0 to (N-1)/N, we get a unitary matrix. 

In [ ]:
N = 4
ts = np.arange(N) / N
freqs = np.arange(N)
args = np.outer(ts, freqs)
M = np.exp(1j * PI2 * args)
print(M)

To check whether a matrix is unitary, we can compute $M^* M$, which should be the identity matrix:

In [ ]:
def dft(ys):
    N = len(ys)
    M = synthesis_matrix(N)
    amps = M.conj().transpose().dot(ys)
    return amps

And compare it to analyze2:

In [ ]:
print(dft(ys))

The result is close to `amps * 4`.

We can also compare it to `np.fft.fft`.  FFT stands for Fast Fourier Transform, which is an even faster implementation of DFT.

In [ ]:
M = synthesis_matrix(N=8)

And the DFT matrix:

In [ ]:
Mstar = M.conj().transpose()

And a triangle wave with 8 elements:

In [ ]:
X5 = row5.dot(wave.ys)
X5

And they are the same, within floating point error.

In [ ]:
abs(X3 - X5)

Let's try the same thing with a complex signal:

In [ ]:
wave2 = thinkdsp.ComplexSinusoid(freq=1).make_wave(duration=1, framerate=8)
thinkplot.plot(wave2.ts, wave2.ys.real)
thinkplot.plot(wave2.ts, wave2.ys.imag)

Now the elements with indices 3 and 5 are different:

In [ ]:
MstarM = M.conj().transpose().dot(M)
print(MstarM.real)

The result is actually $4 I$, so in general we have an extra factor of $N$ to deal with, but that's a minor problem.

We can use this result to write a faster version of `analyze1`:


In [ ]:
def analyze2(ys, freqs, ts):
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys) / N
    return amps

In [ ]:
N = 4
amps = np.array([0.6, 0.25, 0.1, 0.05])
freqs = np.arange(N)
ts = np.arange(N) / N
ys = synthesize2(amps, freqs, ts)

amps3 = analyze2(ys, freqs, ts)
print(amps3)

Now we can write our own version of DFT:

In [ ]:
def synthesis_matrix(N):
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    return M

In [ ]:
X3 = row3.dot(wave2.ys)
X3

In [ ]:
X5 = row5.dot(wave2.ys)
X5

Visually we can confirm that the FFT of the real signal is symmetric:

In [ ]:
hs = np.fft.fft(wave.ys)
thinkplot.plot(abs(hs))

And the FFT of the complex signal is not.

In [ ]:
hs = np.fft.fft(wave2.ys)
thinkplot.plot(abs(hs))

Another way to think about all of this is to evaluate the DFT matrix for different frequencies.  Instead of $0$ through $N-1$, let's try $0, 1, 2, 3, 4, -3, -2, -1$.

In [ ]:
N = 8
ts = np.arange(N) / N
freqs = np.arange(N)
freqs = [0, 1, 2, 3, 4, -3, -2, -1]
args = np.outer(ts, freqs)
M2 = np.exp(1j * PI2 * args)

In [ ]:
approx_equal(M, M2)

So you can think of the second half of the DFT as positive frequencies that get aliased (which is how I explained them), or as negative frequencies (which is the more conventional way to explain them).  But the DFT doesn't care either way.

The `thinkdsp` library provides support for computing the "full" FFT instead of the real FFT.